# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guarapari,-20.6667,-40.4975,86.90,61,98,6.44,BR,1649513564
1,1,cidreira,-30.1811,-50.2056,71.82,83,97,9.15,BR,1649513564
2,2,kapaa,22.0752,-159.3190,73.47,88,100,20.71,US,1649513509
3,3,kempele,64.9131,25.5034,40.12,52,75,5.75,FI,1649513565
4,4,hilo,19.7297,-155.0900,69.53,83,100,5.75,US,1649513565


In [3]:
len(city_data_df)

558

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create a list of locations that are between 70 and 80 deg with no cloud cover and wind speed less than 10 mph. drop non
# qualifiying locations and create a perfect_place list
perfect_place = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
perfect_place


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
89,89,chuy,-33.6971,-53.4616,74.28,61,0,4.59,UY,1649513573
248,248,kahului,20.8947,-156.4700,74.84,77,0,8.99,US,1649513591
263,263,presidencia roque saenz pena,-26.7852,-60.4388,77.22,67,0,7.94,AR,1649513593
374,374,maldonado,-34.6667,-54.9167,74.59,55,0,4.43,UY,1649513606
390,390,mercedes,-34.6515,-59.4307,74.64,51,0,7.02,AR,1649513607
418,418,rocha,-34.4833,-54.3333,76.87,55,0,3.42,UY,1649513610
445,445,san patricio,28.0170,-97.5169,71.47,56,0,9.46,US,1649513614
446,446,adelaide,-34.9333,138.6000,75.22,43,0,8.05,AU,1649513436
455,455,beihai,21.4833,109.1000,73.17,67,0,6.29,CN,1649513615
488,488,moron,-34.6534,-58.6198,76.01,77,0,4.68,AR,1649513622


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = perfect_place[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
89,chuy,UY,-33.6971,-53.4616,
248,kahului,US,20.8947,-156.4700,
263,presidencia roque saenz pena,AR,-26.7852,-60.4388,
374,maldonado,UY,-34.6667,-54.9167,
390,mercedes,AR,-34.6515,-59.4307,
418,rocha,UY,-34.4833,-54.3333,
445,san patricio,US,28.0170,-97.5169,
446,adelaide,AU,-34.9333,138.6000,
455,beihai,CN,21.4833,109.1000,
488,moron,AR,-34.6534,-58.6198,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

,City,Country,Lat,Lng,Hotel Name
89,chuy,UY,-33.6971,-53.4616,Nuevo Hotel Plaza
248,kahului,US,20.8947,-156.4700,Maui Seaside Hotel
263,presidencia roque saenz pena,AR,-26.7852,-60.4388,Hotel Gualok
374,maldonado,UY,-34.6667,-54.9167,La Cimarrona
390,mercedes,AR,-34.6515,-59.4307,Posada del Inti Centro
418,rocha,UY,-34.4833,-54.3333,Hotel Arrarte
445,san patricio,US,28.0170,-97.5169,"Motel 6 Sinton, TX"
446,adelaide,AU,-34.9333,138.6000,The Chancellor on Currie
455,beihai,CN,21.4833,109.1000,21 Degree Hostel
488,moron,AR,-34.6534,-58.6198,Hotel Morón


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))